<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:


      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 12:40:42
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 1.01 L (0.71%)
Current PnL: -16.54 L (-11.13%)
CY Booked + Current PnL: -5.51 L (-3.71%)
-------------------
Total profit:  2.32 L
Total loss:  -18.86 L
-------------------
Total Booked + Current PnL: 21.47 L (17.64%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.73%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.44 L (57.79%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-9.44,72.0,M-LC,4.33,223223.0,21744.0,5469.0,2.13,10.79,2.45,13.51,60.0,3.98,1.59,24.75,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.20,59.0,H-LC,11.07,173503.0,17326.0,8762.0,0.90,11.09,5.05,16.71,22.0,1.98,1.24,30.47,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.98,62.0,M-SC,4.41,88752.0,-12025.0,12115.0,0.80,-11.93,13.65,0.09,245.0,-0.99,0.63,16.60,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,69.0,H-MC,5.67,114268.0,13764.0,16877.0,-0.55,13.70,14.77,30.49,79.0,0.82,0.81,50.54,MH,ATH,METALS
57,RELIANCE,1533.00,-12.16,58.0,H-LC,5.60,220116.0,9750.0,19040.0,0.84,4.63,8.65,13.68,37.0,0.51,1.57,21.92,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,43.0,H-MC,2.23,135826.0,38.0,42717.0,-0.69,0.03,31.45,31.48,107.0,0.00,0.97,12.15,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.62,63.0,H-MC,1.80,249319.0,1588.0,73474.0,0.19,0.64,29.47,30.30,92.0,0.02,1.77,14.02,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.55,54.0,H-MC,4.38,201120.0,300.0,68481.0,1.20,0.15,34.05,34.25,80.0,0.00,1.43,5.35,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.13,40.0,H-LC,5.59,302189.0,-13653.0,126315.0,0.42,-4.32,41.80,35.67,5.0,-0.11,2.15,8.93,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.32,45.0,H-LC,8.16,218786.0,-32982.0,86202.0,-0.66,-13.10,39.40,21.13,27.0,-0.38,1.56,17.68,X40,ATH,PAINTS
20,CIPLA,1795.00,-20.12,49.0,H-LC,5.80,212884.0,8384.0,33040.0,0.36,4.10,15.52,20.25,10.0,0.25,1.52,13.19,X40N,BTT,PHARMA
42,ITC,452.00,-37.92,51.0,H-LC,2.89,200955.0,817.0,20075.0,-0.41,0.41,9.99,10.44,4.0,0.04,1.43,5.99,X40,NTT,FMCG
44,JIOFIN,387.00,0.55,53.0,H-LC,13.40,273179.0,5814.0,59635.0,0.73,2.17,21.83,24.48,48.0,0.10,1.94,58.34,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-1.73,65.0,H-LC,1.00,243450.0,-2701.0,81921.0,1.28,-1.10,33.65,32.18,16.0,-0.03,1.73,34.55,X200,ATH,IT
42,ITC,452.00,-37.92,51.0,H-LC,2.89,200955.0,817.0,20075.0,-0.41,0.41,9.99,10.44,4.0,0.04,1.43,5.99,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.99,54.0,H-LC,4.10,280238.0,14812.0,41980.0,-1.23,5.58,14.98,21.40,11.0,0.35,1.99,13.07,XY25,NTT,FMCG
84,VBL,671.64,-16.13,40.0,H-LC,5.59,302189.0,-13653.0,126315.0,0.42,-4.32,41.80,35.67,5.0,-0.11,2.15,8.93,X40N,ATH,FMCG
57,RELIANCE,1533.00,-12.16,58.0,H-LC,5.60,220116.0,9750.0,19040.0,0.84,4.63,8.65,13.68,37.0,0.51,1.57,21.92,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.0,64.74,70.0,H-SC,8.50,142647.0,-40128.0,151192.0,6.27,-21.96,105.99,60.77,125.0,-0.27,1.02,27.76,XR,NTT,JEWELLERY
17,CAMPUS,393.0,-22.96,65.0,M-SC,1.45,160458.0,-7317.0,63156.0,3.94,-4.36,39.36,33.28,210.0,-0.12,1.14,24.37,XY24,NTT,FOOTWEAR
56,RELAXO,1176.0,-31.36,59.0,H-SC,1.74,93605.0,-51555.0,127481.0,3.52,-35.52,136.19,52.31,136.0,-0.40,0.67,25.30,X40N,NTT,FOOTWEAR
53,QUESS,986.0,-39.39,55.0,M-SC,40.57,60116.0,-4890.0,155821.0,3.43,-7.52,259.20,232.18,198.0,-0.03,0.43,4.69,XY24,NTT,MISC
77,TRIDENT,48.0,-5.02,65.0,M-SC,5.33,73976.0,-18278.0,44585.0,3.28,-19.81,60.27,28.51,224.0,-0.41,0.53,26.58,XR,NTT,TEXTILES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,37.50,47.0,M-SC,8.42,181658.0,-80560.0,150395.0,-3.37,-30.72,82.79,26.63,229.0,-0.54,1.29,12.95,XY24,NTT,MISC
65,SHALBY,327.00,1061.08,64.0,M-SC,2.54,157803.0,-23340.0,68802.0,-2.16,-12.88,43.60,25.10,235.0,-0.34,1.12,26.46,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-51.47,46.0,H-SC,12.11,135542.0,12206.0,121554.0,-1.98,9.90,89.68,108.45,119.0,0.10,0.96,34.46,AR,ATH,MISC
79,UJJIVANSFB,60.00,96.03,55.0,H-SC,12.42,123768.0,-18711.0,38232.0,-1.88,-13.13,30.89,13.70,163.0,-0.49,0.88,46.08,OX40N,NTT,BANKS
49,MASFIN,398.61,-16.89,53.0,H-SC,12.90,94680.0,-3300.0,24901.0,-1.34,-3.37,26.30,22.05,152.0,-0.13,0.67,37.62,XR,ATH,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.98,62.0,M-SC,4.41,88752.0,-12025.0,12115.0,0.80,-11.93,13.65,0.09,245.0,-0.99,0.63,16.60,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,55.0,H-SC,12.42,123768.0,-18711.0,38232.0,-1.88,-13.13,30.89,13.70,163.0,-0.49,0.88,46.08,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,59.0,H-SC,1.73,223605.0,-46062.0,82398.0,0.93,-17.08,36.85,13.47,138.0,-0.56,1.59,13.10,XY24,NTT,PAINTS
19,CERA,9475.0,-20.05,46.0,H-SC,2.16,145694.0,-30209.0,72235.0,0.25,-17.17,49.58,23.89,149.0,-0.42,1.04,25.83,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.80,108376.0,-23459.0,65687.0,0.33,-17.79,60.61,32.03,98.0,-0.36,0.77,21.39,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,59.0,H-LC,12.77,297989.0,-47967.0,122831.0,1.87,-13.86,41.22,21.64,1.0,-0.39,2.12,5.56,X40,ATH,IT
40,INFY,2275.00,-16.73,55.0,H-LC,8.76,324420.0,11532.0,160166.0,0.97,3.69,49.37,54.87,3.0,0.07,2.31,10.58,X40,BTT,IT
42,ITC,452.00,-37.92,51.0,H-LC,2.89,200955.0,817.0,20075.0,-0.41,0.41,9.99,10.44,4.0,0.04,1.43,5.99,X40,NTT,FMCG
84,VBL,671.64,-16.13,40.0,H-LC,5.59,302189.0,-13653.0,126315.0,0.42,-4.32,41.80,35.67,5.0,-0.11,2.15,8.93,X40N,ATH,FMCG
1,ABB,7934.00,-38.31,64.0,H-LC,11.11,257765.0,-3854.0,123057.0,0.59,-1.47,47.74,45.57,7.0,-0.03,1.83,9.68,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,66.0,M-MC,6.97,229250.0,4288.0,163364.0,0.26,1.91,71.26,74.53,192.0,0.03,1.63,34.77,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,12.39,61.0,H-MC,3.72,193940.0,29270.0,38691.0,2.66,17.77,19.95,41.28,89.0,0.76,1.38,27.18,X40N,ATH,AC
30,HAVELLS,2069.16,0.62,63.0,H-MC,1.80,249319.0,1588.0,73474.0,0.19,0.64,29.47,30.30,92.0,0.02,1.77,14.02,X40,ATH,ELECTRICAL
85,VOLTAS,1918.49,0.46,56.0,H-MC,2.70,210810.0,19068.0,76967.0,-0.73,9.94,36.51,50.08,99.0,0.25,1.50,17.42,XY25,ATH,AC
18,CAMS,5211.76,-2.50,59.0,H-SC,4.41,110790.0,8786.0,35143.0,2.51,8.61,31.72,43.06,122.0,0.25,0.79,27.20,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,69.0,H-MC,5.67,114268.0,13764.0,16877.0,-0.55,13.70,14.77,30.49,79.0,0.82,0.81,50.54,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-40.16,54.0,M-SC,0.67,101040.0,9542.0,69212.0,0.27,10.43,68.50,86.07,223.0,0.14,0.72,46.73,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.47,46.0,H-SC,12.11,135542.0,12206.0,121554.0,-1.98,9.90,89.68,108.45,119.0,0.10,0.96,34.46,AR,ATH,MISC
87,WIPRO,420.00,-10.16,58.0,M-LC,5.54,158081.0,7136.0,102326.0,1.47,4.73,64.73,72.51,53.0,0.07,1.13,11.04,XR,NTT,IT
36,IEX,219.00,-28.31,60.0,H-SC,17.22,205043.0,5089.0,92146.0,2.64,2.55,44.94,48.63,115.0,0.06,1.46,14.39,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1898.74,67.0,L-SC,4.40,54548.0,-82629.0,82722.0,2.40,-60.24,151.65,0.07,267.0,-1.00,0.39,35.66,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.00,-41.65,54.0,L-SC,41.86,86362.0,-27187.0,67241.0,1.26,-23.94,77.86,35.27,268.0,-0.40,0.61,111.37,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.33,50.0,H-SC,13.36,89986.0,-10977.0,105509.0,1.52,-10.87,117.25,93.63,148.0,-0.10,0.64,34.03,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,53.0,H-SC,12.90,94680.0,-3300.0,24901.0,-1.34,-3.37,26.30,22.05,152.0,-0.13,0.67,37.62,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,69.0,H-MC,5.67,114268.0,13764.0,16877.0,-0.55,13.70,14.77,30.49,79.0,0.82,0.81,50.54,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3862.16,64.0,M-SC,6.47,123593.0,-23775.0,75589.0,1.13,-16.13,61.16,35.16,236.0,-0.31,0.88,26.24,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-41.65,54.0,L-SC,41.86,86362.0,-27187.0,67241.0,1.26,-23.94,77.86,35.27,268.0,-0.40,0.61,111.37,XR,NTT,HOTELS
54,RAJESHEXPO,518.0,1898.74,67.0,L-SC,4.40,54548.0,-82629.0,82722.0,2.40,-60.24,151.65,0.07,267.0,-1.00,0.39,35.66,OX40N,NTT,JEWELLERY
27,GREENPANEL,537.0,238.50,67.0,M-SC,4.08,151085.0,-31993.0,110972.0,0.41,-17.48,73.45,43.14,230.0,-0.29,1.08,42.12,XY24,NTT,MISC
76,TITAGARH,1548.0,0.96,69.0,H-SC,4.10,165445.0,-26570.0,105455.0,-0.31,-13.84,63.74,41.08,158.0,-0.25,1.18,40.19,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,42.78
2,50,73.23


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.29
LC    34.04
MC    22.65
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.48
X40      15.00
X40N     12.86
XR       11.42
XY25      9.58
AR        8.51
OX40N     8.02
X200      1.73
MH        1.68
X5K       1.43
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.46
H-LC    25.89
H-MC    19.76
M-SC    13.31
M-LC     7.10
M-MC     2.59
L-SC     1.52
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.22,-13.12,74.81
FMCG,11.57,-4.09,40.51
FINANCE,9.49,-15.33,60.50
BANKS,7.66,-11.41,62.70
MISC,7.34,-11.38,73.32
PAINTS,5.71,-15.15,32.31
ELECTRICAL,5.59,-4.69,62.39
AC,3.58,3.66,28.96
INSURANCE,3.36,-6.78,43.40


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2844125.0,22
XR,1305094.0,14
AR,1298180.0,9
X40,741820.0,10
X40N,608065.0,10
OX40N,520271.0,10
XY25,404933.0,7
SR,243519.0,2
X5K,117112.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3412649.0,29
M-SC,1376935.0,17
H-LC,1237606.0,15
H-MC,1177191.0,15
M-LC,382686.0,5
M-MC,312982.0,2
L-SC,246747.0,3
L-MC,60401.0,1
L-LC,36542.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1029791.0      6
M-SC       XY24       796866.0      7
H-SC       AR         782986.0      5
           XR         754357.0      7
H-MC       XY24       487072.0      4
H-LC       X40        461353.0      5
           AR         354037.0      2
M-MC       XY24       312982.0      2
H-SC       X40N       290326.0      4
           OX40N      249848.0      4
           SR         243519.0      2
H-LC       X40N       219640.0      4
H-MC       X40        211424.0      4
           XY25       187844.0      2
L-SC       XR         164025.0      2
M-SC       AR         161157.0      2
M-LC       XY24       157779.0      2
M-SC       OX40N      122014.0      4
M-LC       X5K        117112.0      1
M-SC       XY25       114058.0      1
           XR         113797.0      2
H-MC       XR         110188.0      1
M-LC       XR         102326.0      1
H-MC       X40N        98099.0      2
L-SC       OX40N       82722.0      1
H-LC       X200        81921.0      1
M-SC       X40         69043.0      1
H-MC       OX40N       65687.0      1
H-SC       MH          61822.0      1
H-LC       XY25        61020.0      2
L-MC       XR          60401.0      1
H-LC       XY24        59635.0      1
L-LC       XY25        36542.0      1
H-MC       MH          16877.0      1
M-LC       XY25         5469.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
